In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Baseline(Gemma) Templete

In [ ]:


model_id = "gg-hf/gemma-2b-it"
dtype = torch.bfloat16

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    torch_dtype=dtype,
)

chat = [
    { "role": "user", "content": "Write a hello world program" },
]
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

# Output
'''
<bos><start_of_turn>user
Write a hello world program<end_of_turn>
<start_of_turn>model
'''

# EXAONE Templete

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct")

# Choose your prompt
prompt = "루이 14세는 유능한 조언자의 도움을 많이 받았는데, 다음 중 그가 프랑스를 재편하고 열강으로 만드는 데 도움을 준 조언자는 누구였습니까?"   # Korean example

messages = [
    {"role": "system", 
     "content": "You are EXAONE model from LG AI Research, a helpful assistant."},
    {"role": "user", "content": prompt}
]
input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)
output = model.generate(
    input_ids.to("cuda"),
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=128
)
print(tokenizer.decode(output[0]))
# prompt = tokenizer.apply_chat_template(
#     messages,
#     tokenize=False,
#     add_generation_prompt=True,
# )
# prompt
# '''
# [|system|]You are EXAONE model from LG AI Research, a helpful assistant.[|endofturn|]
# [|user|]너의 소원을 말해봐
# [|assistant|]
# '''

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

[|system|]You are EXAONE model from LG AI Research, a helpful assistant.[|endofturn|]
[|user|]루이 14세는 유능한 조언자의 도움을 많이 받았는데, 다음 중 그가 프랑스를 재편하고 열강으로 만드는 데 도움을 준 조언자는 누구였습니까?
[|assistant|]루이 14세의 통치 기간 동안 프랑스를 재편하고 강대국으로 만드는 데 중요한 역할을 한 주요 조언자는 장 바티스트 콜베르(Jean-Baptiste Colbert)였습니다. 콜베르는 재무장관으로서 경제 정책을 주도하며 중상주의를 추진했고, 이를 통해 프랑스의 경제적 기반을 강화했습니다. 그의 노력 덕분에 프랑스는 17세기 말과 18세기 초에 걸쳐 유럽의 주요 강국 중 하나로 부상할 수 있었습니다.[|endofturn|]


In [7]:
tokenizer.special_tokens_map

{'bos_token': '[BOS]',
 'eos_token': '[|endofturn|]',
 'unk_token': '[UNK]',
 'pad_token': '[PAD]'}

In [2]:
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
# 모델 불러오기
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="beomi/Solar-Ko-Recovery-11B",
    max_seq_length=1024,
    dtype=None,
    load_in_4bit=True,
)

==((====))==  Unsloth 2024.11.7: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla V100-SXM2-32GB. Max memory: 31.739 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu124. CUDA = 7.0. CUDA Toolkit = 12.4.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


beomi/Solar-Ko-Recovery-11B does not have a padding token! Will use pad_token = <unk>.


In [7]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

In [8]:
tokenizer.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'pad_token': '</s>'}

In [9]:
tokenizer.pad_token_id

2

In [10]:
tokenizer("<start_of_turn>model")

{'input_ids': [1, 523, 2521, 28730, 1009, 28730, 499, 28767, 3549], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}